In [1]:
import spotipy
import billboard
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids
ids = getTrackIDs('Billboard', '6UeSakyzhiEt4NB3UAd6NQ')
print(len(ids))

100


In [4]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

#ids2 = getTrackIDsFromAlbum('2Y9IRtehByVkegoD7TcLfi')
ids2 = getTrackIDsFromAlbum('0RQwQQSoR4E74jfT47ATpC')
print(len(ids2))
print(ids2)

20
['7LJeCI3v09UtUq3ETKKpgX', '5ypnp8Gi7I9duNOHaWks1r', '6ueA3RMronnNPEGgURsn2S', '4UyqE0A2NfKkPYLTTKTDqL', '6vAB19HadQGdU8yyvKRihs', '42DuSc89cXlRFzaP99W0hp', '1fFUu73Zyida3IzpNsJly9', '4pCI9LrUmH1AbIPlHPIe96', '5F38kOsptiPOVpBidD31Jz', '5tI3QwCrI0B0UYdNv7UEqa', '1wnqPz25idlXzEgI9J0hYe', '7L5ClYxMAZlpM08HkU28ro', '5es0F8fWeGdG3v7AAq6dMs', '0CkVEkQjA7tf0aGJ1UKopa', '2pUlpG7uiDk7evGOgbOC30', '4wwr9OBUij06t2BvH8yeBI', '3PildhaquXx0YNtEywZQhE', '6SAyAS35hL10OeL0jY89hS', '3IF6W3Garp7Ewfhdc28pn9', '5n1h3oQb2jKmIgOccDkuOQ']


In [5]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [6]:
def getTrackIdFromBillboard(name):
    results = sp.search(q="track:" + name)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [7]:
def getChartDataFrame(chart):
    start1 = time.process_time()
    tracks = []
    for entry in chart.entries:
        id = getTrackIdFromBillboard(entry.title)
        if (id != "NONE"):
            track = getTrackFeatures(id)
            tracks.append(track)
    end1 = time.process_time()
    print("time to get billboard_chart: " + str(end1 - start1))
    start2 = time.process_time()
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
    end2 = time.process_time()
    print("time to insert tracks into dataframe: " + str(end2 - start2))
    return df


In [9]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    charts = []
    newDataFrame = pd.DataFrame()
    while (chart.previousDate):
        chart = billboard.ChartData('hot-100', chart.previousDate)
        dataFrame = getChartDataFrame(chart)
        newDataFrame = newDataFrame.append(dataFrame)
        i += 1
        if (i == 1):
            break
    return newDataFrame

In [10]:
start1 = time.process_time()
start2 = time.clock()
dataFrame = gatherBillboardData()
end1 = time.process_time()
end2 = time.clock()

elapsed1 = end1 - start1
elapsed2 = end2 - start2
print(len(dataFrame))
print("total time elapsed using process_time: " + str(elapsed1))
print("total time elapsed using clock:        " + str(elapsed2))





C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


time to get billboard_chart: 2.671875
time to insert tracks into dataframe: 0.0
99
total time elapsed using process_time: 4.609375
total time elapsed using clock:        33.52912070000002


C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


In [ ]:
start = time.process_time()
morn = dataFrame.drop_duplicates()
end = time.process_time()
print(len(morn))
print("elapsed time was: " + str(end - start))

In [11]:
dataFrame.head()

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Savage Love (Laxed - Siren Beat),Savage Love (Laxed - Siren Beat),Jawsh 685,2020-06-11,171374,93,0.767,0.2340,0.767,0.481,0.000000,0.2690,-8.520,0.0803,150.076,4
1,Dynamite,Dynamite (DayTime Version),BTS,2020-08-28,199053,97,0.746,0.0112,0.746,0.765,0.000000,0.0936,-4.410,0.0993,114.044,4
2,WAP (feat. Megan Thee Stallion),WAP (feat. Megan Thee Stallion),Cardi B,2020-08-07,187541,98,0.935,0.0194,0.935,0.454,0.000000,0.0824,-7.509,0.3750,133.073,4
3,Mood (feat. iann dior),Mood (feat. iann dior),24kGoldn,2020-07-24,140525,100,0.700,0.2210,0.700,0.722,0.000000,0.2720,-3.558,0.0369,90.989,4
4,Laugh Now Cry Later (feat. Lil Durk),Laugh Now Cry Later (feat. Lil Durk),Drake,2020-08-14,261492,95,0.761,0.2440,0.761,0.518,0.000035,0.1070,-8.871,0.1340,133.976,4
